In [1]:
kernel.silent(true)

In [2]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [3]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang
import reflect.io._, Path._
import reflect.io._
import Path._
import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._
kernel.silent(false)

In [4]:
val uri = (value: String) => TripleItem.Uri(value)
val qbDataSet = "http://purl.org/linked-data/cube#dataSet";

uri: String => TripleItem.Uri = ammonite.$sess.cmd3$Helper$$Lambda$2799/0x0000000840c6f840@746b6b3
qbDataSet: String = "http://purl.org/linked-data/cube#dataSet"

In [5]:
val graphs = "../data/jaur".toDirectory.files.map(_.path).filter(g => g matches """.*\.ttl""")

graphs: Iterator[String] = non-empty iterator

In [6]:
var jaurCubesMap: Map[String, Graph] = Map()
graphs.foreach(g => {
    val name = g.replaceAll("\\.ttl", "").replaceAll("^.*/", "")
    jaurCubesMap = jaurCubesMap + (name -> Graph("czso",g))
})

jaurCubesMap: Map[String, Graph] = Map(
  "jaur-regions-bysex" -> com.github.propi.rdfrules.data.Graph@cc371fa,
  "jaur-regions-total" -> com.github.propi.rdfrules.data.Graph@270a3ad5,
  "jaur-districts-total" -> com.github.propi.rdfrules.data.Graph@64eddec3,
  "jaur-districts-bysex" -> com.github.propi.rdfrules.data.Graph@5157a832
)

In [7]:
jaurCubesMap.foreach(r => println(r._1 + ": " + r._2.filter(t => t.predicate.hasSameUriAs(qbDataSet)).size))

jaur-regions-bysex: 234
jaur-regions-total: 117
jaur-districts-total: 684
jaur-districts-bysex: 1368


In [8]:
val intervalCounts = Array(10,30,50,100)
val supportPercents = Array(1,2,3)
val equiSize = (support: Int) => DiscretizationTask.Equisize(support)
val equiFrequent = (intervalCount: Int) => DiscretizationTask.Equifrequency(intervalCount)

intervalCounts: Array[Int] = Array(10, 30, 50, 100)
supportPercents: Array[Int] = Array(1, 2, 3)
equiSize: Int => DiscretizationTask.Equisize = ammonite.$sess.cmd7$Helper$$Lambda$3032/0x0000000840dfd840@273c7de2
equiFrequent: Int => DiscretizationTask.Equifrequency = ammonite.$sess.cmd7$Helper$$Lambda$3033/0x0000000840dfe040@594f6c25

In [9]:
import eu.easyminer.discretization.impl.Interval
import eu.easyminer.discretization.impl.IntervalBound._
val rounded = (value: Double, scale: Integer) => BigDecimal(value).setScale(scale, BigDecimal.RoundingMode.HALF_UP).toDouble
val intervalToString = (i: Interval) => "<"+rounded(i.minValue.value,2)+ "__"+rounded(i.maxValue.value,2)+")"

import eu.easyminer.discretization.impl.Interval

import eu.easyminer.discretization.impl.IntervalBound._

rounded: (Double, Integer) => Double = ammonite.$sess.cmd8$Helper$$Lambda$3058/0x0000000840e0b040@1bfca1ec
intervalToString: Interval => String = ammonite.$sess.cmd8$Helper$$Lambda$3059/0x0000000840e0b840@2cc63a0a

In [10]:
// in all cubes
val czsoUri = "http://data.czso.cz/ontology/"
val unemploymentRate = uri(czsoUri+"podilNezamestnanych")
val reachableApplicants = uri(czsoUri+"dosazitelniNeumisteniUchazeciOZamestnani")

// only in total cubes
val unplacedApplicants = uri(czsoUri+"neumisteniUchazeciOZamestnani")
val vacaniesCount = uri(czsoUri+"pocetVolnychMist")

val measureUris = Array(unemploymentRate,reachableApplicants,unplacedApplicants,vacaniesCount) 

czsoUri: String = "http://data.czso.cz/ontology/"
unemploymentRate: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/podilNezamestnanych"
)
reachableApplicants: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/dosazitelniNeumisteniUchazeciOZamestnani"
)
unplacedApplicants: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/neumisteniUchazeciOZamestnani"
)
vacaniesCount: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/pocetVolnychMist"
)
measureUris: Array[TripleItem.Uri] = Array(
  LongUri("http://data.czso.cz/ontology/podilNezamestnanych"),
  LongUri(
    "http://data.czso.cz/ontology/dosazitelniNeumisteniUchazeciOZamestnani"
  ),
  LongUri("http://data.czso.cz/ontology/neumisteniUchazeciOZamestnani"),
  LongUri("http://data.czso.cz/ontology/pocetVolnychMist")
)

In [11]:
val discretizeEquisize = (graph: Graph, relativeSupport: Int, measure: TripleItem.Uri) => {
    val observationsCount = graph.filter(t => t.predicate.hasSameUriAs(qbDataSet)).size
    val absoluteSupport = (observationsCount * (relativeSupport.toFloat / 100)).ceil.toInt
    graph.filter(t => t.predicate == measure)
         .discretize(equiSize(absoluteSupport))(quad => quad.triple.predicate == measure)
}

val discretizeEquifrequent = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretize(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

val discretizeEquisizeAndGetIntervals = (graph: Graph, relativeSupport: Int, measure: TripleItem.Uri) => {
    val observationsCount = graph.filter(t => t.predicate.hasSameUriAs(qbDataSet)).size
    val absoluteSupport = (observationsCount * (relativeSupport.toFloat / 100)).ceil.toInt
    graph.filter(t => t.predicate == measure)
         .discretizeAndGetIntervals(equiSize(absoluteSupport))(quad => quad.triple.predicate == measure)   
}

val discretizeEquifrequentAndGetIntervals = (graph: Graph, intervalsCount: Int, measure: TripleItem.Uri) => {
    graph.filter(t => t.predicate == measure)
         .discretizeAndGetIntervals(equiFrequent(intervalsCount))(quad => quad.triple.predicate == measure)
}

discretizeEquisize: (Graph, Int, TripleItem.Uri) => Graph = ammonite.$sess.cmd10$Helper$$Lambda$3098/0x0000000840e2f840@7871ecd2
discretizeEquifrequent: (Graph, Int, TripleItem.Uri) => Graph = ammonite.$sess.cmd10$Helper$$Lambda$3099/0x0000000840e2fc40@42604dfd
discretizeEquisizeAndGetIntervals: (Graph, Int, TripleItem.Uri) => IndexedSeq[Interval] = ammonite.$sess.cmd10$Helper$$Lambda$3100/0x0000000840e2f040@5b1e3963
discretizeEquifrequentAndGetIntervals: (Graph, Int, TripleItem.Uri) => IndexedSeq[Interval] = ammonite.$sess.cmd10$Helper$$Lambda$3101/0x0000000840e2f440@78e937bb

In [12]:
val lowerBound = (interval: String) => interval.split(" ")(1)
val upperBound = (interval: String) => interval.split(" ")(3)
val lowerBoundInterval = (interval: Interval) => interval.minValue.value.toString
val upperBoundInterval = (interval: Interval) => interval.maxValue.value.toString

val equalIntervals = (i1: String, i2: Interval) => {
    lowerBound(i1).equals(lowerBoundInterval(i2)) && upperBound(i1).equals(upperBoundInterval(i2))
}

lowerBound: String => String = ammonite.$sess.cmd11$Helper$$Lambda$3136/0x0000000840e49840@e2d068e
upperBound: String => String = ammonite.$sess.cmd11$Helper$$Lambda$3137/0x0000000840e4a040@6a7e618f
lowerBoundInterval: Interval => String = ammonite.$sess.cmd11$Helper$$Lambda$3138/0x0000000840e4b040@5ee4f9d7
upperBoundInterval: Interval => String = ammonite.$sess.cmd11$Helper$$Lambda$3139/0x0000000840e4b840@44eda03c
equalIntervals: (String, Interval) => Boolean = ammonite.$sess.cmd11$Helper$$Lambda$3140/0x0000000840e4c840@7f6cee88

In [13]:
val renameInterval = (t: Triple, intervals: IndexedSeq[Interval], suffix: String) => {
    var result = t
    intervals.foreach(interval => {
        if (equalIntervals(t.`object`.toString, interval)) {
            result = t.copy(`object` = TripleItem.Uri(intervalToString(interval)+"_"+suffix+"_"+(intervals.indexOf(interval)+1)+"/"+intervals.size) )
        }
    })
    result
}

val discretize = (graph: Graph, name: String) => {
    var result = Dataset() + 
    graph.filter(t => !measureUris.contains(t.predicate))
    .map(t => if (t.predicate.hasSameUriAs(qbDataSet)) t.copy(`object` = uri(name)) else t)
    
        measureUris.foreach(measure => {
            intervalCounts.foreach(intervalCount => {
            val intervals = discretizeEquifrequentAndGetIntervals(graph, intervalCount, measure)
            val measures = discretizeEquifrequent(graph, intervalCount, measure)
            result = result + measures.map(t => {renameInterval(t, intervals, "ef"+intervalCount)})
        })

        supportPercents.foreach(percentage => {
            val intervals = discretizeEquisizeAndGetIntervals(graph, percentage, measure)
            val measures = discretizeEquisize(graph, percentage, measure)
            result = result + measures.map(t => {renameInterval(t, intervals, "es"+percentage)})
        })
    })
    result
}

renameInterval: (Triple, IndexedSeq[Interval], String) => Triple = ammonite.$sess.cmd12$Helper$$Lambda$3179/0x0000000840e62c40@3d3b68df
discretize: (Graph, String) => Dataset = ammonite.$sess.cmd12$Helper$$Lambda$3180/0x0000000840e64040@42d86300

In [15]:
jaurCubesMap.foreach(d => discretize(d._2,d._1).cache("../cache/jaur/"+d._1+".cache").export("../exports/jaur/"+d._1+".ttl"))